In [2]:
# 크롤링 밑 지식기반 생성

from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
# 지식 기반 벡터 스토어 저장, 단 여기서는 로컬 GPT를 사용
from langchain_chroma import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, 
                                    embedding=GPT4AllEmbeddings(gpt4all_kwargs = {'allow_download': 'True'}))

In [4]:
question = "what are the approaches to task decomposition"
docs = vectorstore.similarity_search(question)
len(docs)

4

In [8]:
# local llm 불러오기 여기서는 GPT4ALL 사용

from langchain_community.llms import GPT4All
from langchain_core.callbacks import BaseCallbackHandler

count = 0
class MyCustomHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs) -> None:
        global count
        if count < 10:
            print(f"Token: {token}")
            count += 1


# Verbose is required to pass to the callback manager
llm = GPT4All(model=r"c:\Users\kmidd\gpt4all\resources\Meta-Llama-3-8B-Instruct.Q4_0.gguf", callbacks=[MyCustomHandler()], 
              streaming=True, 
              device="cuda",
              max_tokens=2048)

In [9]:
llm.invoke("Simulate a rap battle between Stephen Colbert and John Oliver")

: 